# Antibiotic Indication Processig - Base BERT

Pre-requisites:

1. Set Runtime --> Change Runtime type --> GPUs (or TPUs)
2. Copy the data file (indications, labeled) into `/content/`
3. Set the model type

Install and load libraries

In [1]:
# --- Check for Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

# --- Install packages
if IN_COLAB:
  %pip install -q transformers transformers[torch] datasets
  %pip install -q sacremoses


# --- Load libraries
# Standard libraries
import os

# DS libs
import datasets
import numpy as np
import pandas as pd
import torch

from pathlib import Path
from tqdm.notebook import tqdm

from datasets import Dataset

from transformers import AutoTokenizer, EvalPrediction, pipeline, \
  AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers.pipelines.pt_utils import KeyDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, average_precision_score

In [2]:
# Mounting Google Drive
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  print("Mounted Google Drive at /content/drive")
else:
  print("Current work directory:", os.getcwd())

Current work directory: /home/kevin/DPhil/Projects/EHR-Indication-Processing/02_Models/02_BERTs/Base_BERT


In [3]:
if torch.cuda.is_available():  
  cuda_device = "cuda:0" 
else:  
  cuda_device = "cpu" 

print(f'Using device: {cuda_device}')

!nvidia-smi

Using device: cuda:0
Mon Apr 15 22:45:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro GV100                   Off |   00000000:65:00.0  On |                  Off |
| 29%   39C    P2             29W /  250W |    5279MiB /  32768MiB |     26%      Default |
|                                         |                        |                  N/A |
+--------------------------

## Specify Parameters

In [4]:
# --- Model parameters
model_selection = "Base_BERT"

model_dict = {
    "Base_BERT": "bert-base-uncased",
    "Bio_ClinicalBERT": "emilyalsentzer/Bio_ClinicalBERT",
    
}

# --- Paths
# Base data path
base_data_path = Path("../../../00_Data/")
# Dataset Path (training, testing, etc.)
dataset_path =  base_data_path / "publication_ready"
# Export Path (model checkpoints, predictions, etc.)
export_path = base_data_path / "model_output" / model_selection

assert base_data_path.is_dir(),\
  f"{base_data_path} either doesn't exist or is not a directory."
export_path.mkdir(exist_ok=True)


# --- Misc settings
seed = 42
# Model names
model_name_display = model_selection
model_hf_id = model_dict[model_selection]  # Huggingface name/identifier

# Saved model name: <model_display_name>_<training_set_size>.hf
saved_model_name = f"{model_name_display}_4000.hf"



## Import and check data
Import the training, internal and external test sets & check the splits and data

In [5]:
# Import data --> upload into "Files" on the left-hand panel
train_eval_df = pd.read_csv(
    dataset_path / 'training_oxford_2023-08-23.csv',
    dtype={"Indication": str},
    keep_default_na=False,
    na_values=["NA"],
)

test_oxford_df = pd.read_csv(
    dataset_path / 'testing_oxford_2023-08-23.csv',
    dtype={"Indication": str},
    keep_default_na=False,
    na_values=["NA"],
)

test_banbury_df = pd.read_csv(
    dataset_path / 'testing_banbury_2023-08-23.csv',
    dtype={"Indication": str},
    keep_default_na=False,
    na_values=["NA"],
)

House keeping:
- Split the training data into training & evaluation

- Convert the data into a Huggingface dataset

- Create some nice labels and label<->ID mappers

In [6]:
# --- Split into train and eval
train_df, eval_df = train_test_split(
    train_eval_df, 
    test_size=0.15,
    random_state=seed,
    shuffle=True)

print("Data set size overview:")
print(f"- Training set: {train_df.shape[0]}")
print(f"- Evaluation set: {eval_df.shape[0]}")
print(f"- Testing Oxford set: {test_oxford_df.shape[0]}")
print(f"- Testing Banbury set: {test_banbury_df.shape[0]}")
print()

# check dtypes
#print(train.dtypes)
#print(test.dtypes)

# --- Turn into Huggingface datasets
indication_data = datasets.DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "eval": Dataset.from_pandas(eval_df, preserve_index=False),
    "test": Dataset.from_pandas(test_oxford_df, preserve_index=False),
    "test_external": Dataset.from_pandas(test_banbury_df, preserve_index=False),
})

# --- Create some usable labels
labels = [label for label in indication_data['train'].features.keys() if label not in ['Indication']]
labels_pretty = [" ".join(word.capitalize() for word in label.split("_")) for label in labels]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

Data set size overview:
- Training set: 3400
- Evaluation set: 600
- Testing Oxford set: 2000
- Testing Banbury set: 2000



Calculate class distribution across the sets

In [7]:
class_distribution_dict = {}

for dataset_name, dataset_hf in indication_data.items():
   dataset_pd = dataset_hf.to_pandas()[labels]
   class_distribution_dict[dataset_name] = dataset_pd.sum() / len(dataset_pd) * 100

pd.DataFrame(class_distribution_dict)

,train,eval,test,test_external
urinary,9.088235,7.500000,8.00,15.25
respiratory,15.000000,13.833333,18.70,35.30
abdominal,15.411765,13.333333,11.95,5.90
neurological,3.323529,2.666667,1.15,0.90
skin_soft_tissue,13.323529,13.500000,6.55,9.25
ent,3.500000,3.333333,2.55,0.70
orthopaedic,4.588235,4.333333,2.75,1.20
other_specific,7.794118,9.833333,4.40,1.25
no_specific_source,31.411765,34.000000,45.30,32.50
prophylaxis,21.764706,22.666667,35.20,17.95


In [8]:
# Example datapoint - Indication and labels
print("Example datapoint:")
indication_data['train'][0]

Example datapoint:


{'Indication': '?cellulitis',
 'urinary': 0,
 'respiratory': 0,
 'abdominal': 0,
 'neurological': 0,
 'skin_soft_tissue': 1,
 'ent': 0,
 'orthopaedic': 0,
 'other_specific': 0,
 'no_specific_source': 0,
 'prophylaxis': 0,
 'uncertainty': 1,
 'not_informative': 0}

## Preprocess data
Using AutoTokenizer.

NB: labels needs to be *floats*, not integers for PyTorch to work.

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_hf_id)

def encode_data(data):
  # take a batch of texts
  text = data["Indication"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: data[k] for k in data.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [10]:
encoded_dataset = indication_data.map(encode_data, batched=True, remove_columns=indication_data['train'].column_names)

Map:   0%|          | 0/3400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [12]:
tokenizer.decode(example['input_ids'])

'[CLS]? cellulitis [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [13]:
example['labels']

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['skin_soft_tissue', 'uncertainty']

In [15]:
# Format dataset for PyTorch compatibility 
encoded_dataset.set_format("torch")

## Define model
We load a pre-trained model from the Hugginface portal/hub and fintetune it for our task.

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_hf_id, 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set hyperparameters

In [17]:
# --- Training arguments
batch_size = 8
metric_name = "f1"
leanring_rate = 2e-5
num_train_epochs = 5
weight_decay=0.01

Set training arguments (for Hugging face trainer API) 

In [18]:
# Set up training arguments
args = TrainingArguments(
    output_dir=str(export_path/"training_output"),
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=leanring_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed=seed,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Define multi label metrics

In [19]:
# Source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # First, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # Next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # Finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, probs, average = 'weighted')
    accuracy = accuracy_score(y_true, y_pred)
    # Return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Define training and eval set & start the training

In [20]:
# Setup trainer & run training
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/kevin/DPhil/Projects/EHR-Indication-Processing/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/2125 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.21803532540798187, 'eval_f1': 0.5645470513198687, 'eval_roc_auc': 0.8846822053118208, 'eval_accuracy': 0.35, 'eval_runtime': 1.8953, 'eval_samples_per_second': 316.571, 'eval_steps_per_second': 39.571, 'epoch': 1.0}
{'loss': 0.2979, 'grad_norm': 0.8947088718414307, 'learning_rate': 1.5294117647058822e-05, 'epoch': 1.18}


  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.10612314194440842, 'eval_f1': 0.8320611337729067, 'eval_roc_auc': 0.9682819913102609, 'eval_accuracy': 0.7166666666666667, 'eval_runtime': 1.9161, 'eval_samples_per_second': 313.13, 'eval_steps_per_second': 39.141, 'epoch': 4.0}
{'loss': 0.0861, 'grad_norm': 1.1769434213638306, 'learning_rate': 1.1764705882352942e-06, 'epoch': 4.71}


  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.10176999121904373, 'eval_f1': 0.8582596792134712, 'eval_roc_auc': 0.9685859963641356, 'eval_accuracy': 0.75, 'eval_runtime': 1.8991, 'eval_samples_per_second': 315.942, 'eval_steps_per_second': 39.493, 'epoch': 5.0}
{'train_runtime': 195.2962, 'train_samples_per_second': 87.047, 'train_steps_per_second': 10.881, 'train_loss': 0.1585790723912856, 'epoch': 5.0}


TrainOutput(global_step=2125, training_loss=0.1585790723912856, metrics={'train_runtime': 195.2962, 'train_samples_per_second': 87.047, 'train_steps_per_second': 10.881, 'train_loss': 0.1585790723912856, 'epoch': 5.0})

Evaluate on eval set and get all metrics

In [21]:
# Evaluate on training set
trainer.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.10176999121904373,
 'eval_f1': 0.8582596792134712,
 'eval_roc_auc': 0.9685859963641356,
 'eval_accuracy': 0.75,
 'eval_runtime': 1.945,
 'eval_samples_per_second': 308.491,
 'eval_steps_per_second': 38.561,
 'epoch': 5.0}

Save the weights

In [22]:
trainer.save_model(export_path/saved_model_name)

## Evaluate the model on test sets

Fetch predictions on the test set & calculate metrics

In [23]:
# Inference function
def run_inference(trainer, input_data, threshold=0.5):
    predictions_raw = trainer.predict(input_data).predictions

    # Apply sigmoid function & binarise
    sigmoid = torch.nn.Sigmoid()
    predictions_probs = sigmoid(torch.Tensor(predictions_raw))
    predictions_binarised = (predictions_probs > threshold) * 1

    return(predictions_probs, predictions_binarised)


# Metrics function
def calculate_metrics(y_true, predictions_probs,
                      predictions_binarised,
                      labels, 
                      result_precision=2, 
                      averaging_method = "weighted",
    ):
    # Calculate per class metrics
    scores_per_class = {}
    scores_per_class["F1-Score"] = f1_score(y_true=y_true, y_pred=predictions_binarised, average=None)
    scores_per_class["ROC AUC"] = roc_auc_score(y_true=y_true, y_score=predictions_probs, average=None)
    scores_per_class["PR AUC"] = average_precision_score(y_true=y_true, y_score=predictions_probs, average=None)

    scores_per_class = pd.DataFrame.from_dict(scores_per_class,orient='index', columns=labels)
    
    # Calculate average metrics
    scores_average = {}
    scores_average["F1-Score"] = f1_score(y_true=y_true, y_pred=predictions_binarised, average=averaging_method)
    scores_average["ROC AUC"] = roc_auc_score(y_true=y_true, y_score=predictions_probs, average=averaging_method)
    scores_average["PR AUC"] = average_precision_score(y_true=y_true, y_score=predictions_probs, average=averaging_method)

    # Format into printable string
    metrics_string = ""
    for score_name, avg_score_value in scores_average.items():
        avg_score = avg_score_value.round(result_precision)
        min_sore = scores_per_class.loc[score_name].min().round(result_precision)
        max_score = scores_per_class.loc[score_name].max().round(result_precision) 
        metrics_string += f"{score_name}: {avg_score} ({min_sore}-{max_score})\n"
    
    return scores_per_class, scores_average, metrics_string

Internal & External test set
- Run infererrence on the internal and external test sets
- Calculate per class and average metrics
- Print string in this format per metric `<average>(<worst performing class> - <highest scoring class>)`

In [24]:
pred_threshold = 0.5

test_set_names = {
    "Oxford": "test",
    "Banbury": "test_external"
}

for test_location, test_name in test_set_names.items():
    print("Test set:", test_location)

    # Run inference on test set
    predictions_probs, predictions_binarised = \
        run_inference(trainer, encoded_dataset[test_name], threshold=pred_threshold)

    # Get true labels
    y_test_true = encoded_dataset[test_name]["labels"]

    # Calculate metrics
    scores_per_class, scores_average, metrics_string = \
        calculate_metrics(y_test_true, predictions_probs, predictions_binarised, labels, averaging_method="weighted")
    
    # Print metrics
    pd.set_option('display.precision', 2)
    print(scores_per_class)
    print(metrics_string)

    # Save predictions for further analysis
    predictions_binarised_df = pd.DataFrame(predictions_binarised, columns=labels)
    predictions_binarised_df.to_csv(export_path/f"predictions_{model_selection}_{test_location}.csv", index=False)

    predictions_probs_df = pd.DataFrame(predictions_probs, columns=labels)
    predictions_probs_df.to_csv(export_path/f"predictions_proba_{model_selection}_{test_location}.csv", index=False)


Test set: Oxford


  0%|          | 0/250 [00:00<?, ?it/s]

          urinary  respiratory  abdominal  neurological  skin_soft_tissue  \
F1-Score     0.97         0.98       0.94          0.23              0.94   
ROC AUC      0.98         1.00       0.98          0.97              0.99   
PR AUC       0.98         0.99       0.97          0.69              0.95   

           ent  orthopaedic  other_specific  no_specific_source  prophylaxis  \
F1-Score  0.27         0.59            0.63                0.98         0.98   
ROC AUC   0.96         0.99            0.91                0.99         0.99   
PR AUC    0.85         0.94            0.77                0.99         0.99   

          uncertainty  not_informative  
F1-Score         0.95             0.67  
ROC AUC          1.00             0.98  
PR AUC           0.97             0.92  
F1-Score: 0.93 (0.23-0.98)
ROC AUC: 0.99 (0.91-1.0)
PR AUC: 0.97 (0.69-0.99)

Test set: Banbury


  0%|          | 0/250 [00:00<?, ?it/s]

          urinary  respiratory  abdominal  neurological  skin_soft_tissue  \
F1-Score     0.99         0.99       0.92          0.76              0.96   
ROC AUC      1.00         1.00       0.98          1.00              0.99   
PR AUC       1.00         1.00       0.96          1.00              0.98   

           ent  orthopaedic  other_specific  no_specific_source  prophylaxis  \
F1-Score  0.73         0.63            0.65                0.98         0.98   
ROC AUC   0.99         1.00            0.95                1.00         0.99   
PR AUC    0.90         0.88            0.75                0.98         0.99   

          uncertainty  not_informative  
F1-Score         0.99             0.79  
ROC AUC          1.00             0.97  
PR AUC           0.99             0.91  
F1-Score: 0.97 (0.63-0.99)
ROC AUC: 0.99 (0.95-1.0)
PR AUC: 0.98 (0.75-1.0)

